## **1) Importing Python Packages for GAN**


In [33]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers import LeakyReLU  # Memperbaiki import ini
from tensorflow.keras.optimizers import Adam

import numpy as np
import os

# Membuat direktori untuk gambar yang dihasilkan
os.makedirs('generated_images', exist_ok=True)


## **2) Variables for Neural Networks & Data**

In [34]:
img_width = 28
img_height = 28
channels = 1
img_shape = (img_width, img_height, channels)
latent_dim = 100

# Membuat optimizer Adam dengan learning_rate yang benar
adam = Adam(learning_rate=0.0001)

## **3) Building Generator**





In [35]:
def build_generator():
  model = Sequential()

  model.add(Dense(256, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))
  
  model.summary()
  return model

generator = build_generator()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 256)            │        25,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_12 (LeakyReLU)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_9           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_13 (LeakyReLU)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_14 (LeakyReLU)      │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 784)            │       201,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_3 (Reshape)             │ (None, 28, 28, 1)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 362,000 (1.38 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 1,536 (6.00 KB)

## **4) Building Discriminator**

In [36]:
def build_discriminator():
  model = Sequential()

  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_3 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_15 (LeakyReLU)      │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 533,505 (2.04 MB)

 Trainable params: 533,505 (2.04 MB)

 Non-trainable params: 0 (0.00 B)

## **5) Connecting Neural Networks to build GAN**

In [37]:
GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer=adam)
GAN.summary()


Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_9 (Sequential)       │ (None, 28, 28, 1)      │       362,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_10 (Sequential)      │ (None, 1)              │       533,505 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 895,505 (3.42 MB)

 Trainable params: 360,464 (1.38 MB)

 Non-trainable params: 535,041 (2.04 MB)

## **6) Outputting Images**


In [38]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)
    global save_name
    save_name += 0.00000001
    print("%.8f" % save_name)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            # axs[i,j].imshow(gen_imgs[cnt])
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("generated_images/%.8f.png" % save_name)
    print('saved')
    plt.close()

## **7) Training GAN**

In [39]:
def train(epochs, batch_size=64, save_interval=200):
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale data between -1 and 1
    X_train = X_train / 127.5 - 1.

    # Create our Y for our Neural Networks
    valid = np.ones((batch_size, 1))
    fakes = np.zeros((batch_size, 1))

    for epoch in range(epochs):
        # Get Random Batch
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        # Generate Fake Images
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train discriminator
        d_loss_real = discriminator.train_on_batch(imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Inverse y label
        g_loss = GAN.train_on_batch(noise, valid)

        # Perbaiki akses ke g_loss agar sesuai dengan tipe data yang diharapkan
        print("******* %d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100 * d_loss[1], g_loss[0]))

        if (epoch % save_interval) == 0:
            save_imgs(epoch)

train(30000, batch_size=64, save_interval=200)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step  
******* 0 [D loss: 1.316811, acc: 8.59%] [G loss: 1.125750]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 655ms/step
0.00000001
saved
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 1 [D loss: 1.185907, acc: 15.62%] [G loss: 1.122600]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 2 [D loss: 1.179982, acc: 17.03%] [G loss: 1.139635]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
******* 3 [D loss: 1.166504, acc: 17.73%] [G loss: 1.139073]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 4 [D loss: 1.153271, acc: 18.30%] [G loss: 1.133682]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
******* 5 [D loss: 1.145641, acc: 18.13%] [G loss: 1.131230]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
******* 6 [D loss: 1.154801, acc: 17.52%] [G loss: 1.143034]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 7 [D loss: 1.152371, acc: 17.18%] [G loss: 1.143131]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 8 [D loss: 1.153648, acc: 16.65%] [G loss: 1.145648]
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
******* 9 [D lo

KeyboardInterrupt: 

### **8) Making GIF**

In [40]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

C:\Users\Owner\AppData\Local\Temp\ipykernel_16144\1295497862.py:11: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
C:\Users\Owner\AppData\Local\Temp\ipykernel_16144\1295497862.py:13: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
